In [ ]:
# Importing necessary libraries
import os
import json
import plotly.express as px

def process_json_files(directory_path):
    print(f"Processing directory: {directory_path}")
    
    try:
        # Initialize counts
        total_context_len = 0
        total_generation_len = 0

        # Walk through the directory
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            
            # Check if it's a JSON file
            if file_path.endswith('.json'):
                print(f"Processing file: {filename}")
                try:
                    with open(file_path, 'r') as file:
                        data = json.load(file)
                        
                        # Loop over all array items of data["history"]
                        if 'history' in data:
                            for entry in data['history']:
                                if 'promptLogs' in entry:
                                    for promptLog in entry['promptLogs']:
                                        if 'prompt' in promptLog:
                                            prompt_len = len(promptLog['prompt'])
                                            total_context_len += prompt_len
                                            # print(f"Processed prompt: {promptLog['prompt']}")
                                        if 'completion' in promptLog:
                                            completion_len = len(promptLog['completion'])
                                            total_generation_len += completion_len
                                            # print(f"Processed completion: {promptLog['completion']}")
                except json.JSONDecodeError as e:
                    # Ignore JSON decoding errors, just skip this file
                    print(f"Error decoding JSON file {file_path}: {e}")
                except Exception as e:
                    # Catch any other unexpected errors, print the error and skip this file
                    print(f"Error processing file {file_path}: {e}")
        print(f"Total context length: {total_context_len}, Total generation length: {total_generation_len}")
        return total_context_len, total_generation_len
    except FileNotFoundError:
        # Directory not found, return zeros
        print("Directory not found. Returning zeros.")
        return 0, 0
    except Exception as e:
        # Catch any other unexpected errors, print the error and return zeros
        print(f"Error processing directory: {e}")
        return 0, 0

def plot_data(context_length, generation_length):
    print("Plotting data...")
    try:
        # Calculate token counts assuming every 4 characters = 1 token (crude estimation)
        context_tokens = context_length / 4
        generation_tokens = generation_length / 4
        
        # Bar plot for character count using Plotly
        fig = px.bar(x=['Context Length', 'Generation Length'], y=[context_length, generation_length], 
                      color=['blue', 'green'], title='Total Character Counts')
        fig.update_layout(yaxis_title='Character Count')
        fig.update_layout(legend_title_text='Category', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
        fig.data[0].name = 'Context'
        fig.data[1].name = 'Generation'
        fig.show()
        
        # Bar plot for token count using Plotly
        fig = px.bar(x=['Context Tokens', 'Generation Tokens'], y=[context_tokens, generation_tokens], 
                      color=['red', 'yellow'], title='Token Counts (Crude Estimation)')
        fig.update_layout(yaxis_title='Token Count')
        fig.update_layout(legend_title_text='Category', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
        fig.data[0].name = 'Context'
        fig.data[1].name = 'Generation'
        fig.show()
        print("Plotting complete.")
    except Exception as e:
        # Catch any unexpected errors, print the error
        print(f"Error plotting data: {e}")

# Path to the directory containing the JSON files
directory_path = '/Users/tijs/.continue/sessions'

print("Starting script...")
try:
    # Process the files
    context_length, generation_length = process_json_files(directory_path)

    # Plot the result
    plot_data(context_length, generation_length)
    print("Script complete.")
except Exception as e:
    # Catch any unexpected errors, print the error
    print(f"Error: {e}")
